<a href="https://colab.research.google.com/github/aguilargallardo/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Copy_of_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
df[['Date','Time']] = df.created.apply(lambda x: pd.Series(str(x).split(" "))) 

In [4]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,Date,Time
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-24,07:54:24
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-12,12:19:27
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-17,03:26:41
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-18,02:22:02
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-28,01:32:41


In [5]:
values = ['2016-04', '2016-05']
train = df[df['Date'].str.contains('|'.join(values ))]
test = df[df['Date'].str.contains("2016-06")]
train.Date.value_counts()

2016-04-06    912
2016-05-18    836
2016-05-05    819
2016-05-03    756
2016-04-27    754
             ... 
2016-05-29    234
2016-05-30    213
2016-05-01      6
2016-05-31      5
2016-04-01      4
Name: Date, Length: 61, dtype: int64

In [6]:
test.Date.value_counts()

2016-06-21    1182
2016-06-16    1045
2016-06-12     939
2016-06-01     874
2016-06-29     782
2016-06-02     770
2016-06-03     724
2016-06-24     716
2016-06-08     692
2016-06-22     678
2016-06-11     676
2016-06-28     620
2016-06-14     619
2016-06-09     612
2016-06-04     599
2016-06-10     595
2016-06-07     586
2016-06-15     549
2016-06-25     545
2016-06-17     493
2016-06-23     430
2016-06-20     401
2016-06-05     378
2016-06-13     329
2016-06-27     320
2016-06-06     298
2016-06-26     265
2016-06-18     230
2016-06-19      26
Name: Date, dtype: int64

In [0]:
#Y column
target = 'price'
#X Matrix
features = ['bathrooms', 'bedrooms']

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
#import category_encoders as ce 
#encoder = ce.OneHotEncoder(use_cat_names=True)
#X_train = encoder.fit_transform(X_train)

In [0]:
#X_test = encoder.transform(X_test)

In [11]:
X_train

,bathrooms,bedrooms
2,1.0,1
3,1.0,1
4,1.0,4
5,2.0,4
6,1.0,2
...,...,...
49346,1.0,1
49348,1.0,1
49349,1.0,1
49350,1.0,0


In [12]:
X_test

,bathrooms,bedrooms
0,1.5,3
1,1.0,2
11,1.0,1
14,1.0,1
24,2.0,4
...,...,...
49305,1.0,2
49310,1.0,3
49320,1.0,1
49332,1.0,2


In [0]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae =mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} dollars')

Train Error: 818.53 dollars


In [0]:
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def regression_3d(df, x, y, z, num=100, **kwargs):
    """
    Visualize linear regression in 3D: 2 features + 1 target
    
    df : Pandas DataFrame
    x : string, feature 1 column in df
    y : string, feature 2 column in df
    z : string, target column in df
    num : integer, number of quantiles for each feature
    """
    
    # Plot data
    fig = px.scatter_3d(df, x, y, z, **kwargs)
    
    # Fit Linear Regression
    features = [x, y]
    target = z
    model = LinearRegression()
    model.fit(df[features], df[target])    
    
    # Define grid of coordinates in the feature space
    xmin, xmax = df[x].min(), df[x].max()
    ymin, ymax = df[y].min(), df[y].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))
    
    # Make predictions for the grid
    predictions = model.predict(coords)
    Z = predictions.reshape(num, num).T
    
    # Plot predictions as a 3D surface (plane)
    fig.add_trace(go.Surface(x=xcoords, y=ycoords, z=Z))
    
    return fig

In [0]:
# regression_3d(
#     train,
#     x='price', 
#     y='bedrooms', 
#     z='bathrooms', 
#     text='price', 
#     title=''
# )

# Intercepts and Coef

In [25]:
model.intercept_ , model.coef_

(485.71869002322865, array([2072.61011639,  389.3248959 ]))

# Ryans function

In [0]:
# def squared_errors(df, feature, target, m, b):
#     """
#     Visualize linear regression, with squared errors,
#     in 2D: 1 feature + 1 target.
    
#     Use the m & b parameters to "fit the model" manually.
    
#     df : Pandas DataFrame
#     feature : string, feature column in df
#     target : string, target column in df
#     m : numeric, slope for linear equation
#     b : numeric, intercept for linear requation
#     """
    
#     # Plot data
#     fig = plt.figure(figsize=(7,7))
#     ax = plt.axes()
#     df.plot.scatter(feature, target, ax=ax)
    
#     # Make predictions
#     x = df[feature]
#     y = df[target]
#     y_pred = m*x + b
    
#     # Plot predictions
#     ax.plot(x, y_pred)
    
#     # Plot squared errors
#     xmin, xmax = ax.get_xlim()
#     ymin, ymax = ax.get_ylim()
#     scale = (xmax-xmin)/(ymax-ymin)
#     for x, y1, y2 in zip(x, y, y_pred):
#         bottom_left = (x, min(y1, y2))
#         height = abs(y1 - y2)
#         width = height * scale
#         ax.add_patch(Rectangle(xy=bottom_left, width=width, height=height, alpha=0.1))
    
#     # Print regression metrics
#     mse = mean_squared_error(y, y_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y, y_pred)
#     r2 = r2_score(y, y_pred)
#     print('Mean Squared Error:', mse)
#     print('Root Mean Squared Error:', rmse)
#     print('Mean Absolute Error:', mae)
#     print('R^2:', r2)

# MSE, RMSE, R2

In [28]:
mse = mean_squared_error(y_train, y_pred)
mse

1517879.6665142523

In [29]:
rmse = np.sqrt(mse)
rmse

1232.0225917223484

In [30]:
r2 = r2_score(y_train, y_pred)
r2

0.5111543084316607